# Bootcamp Data Science y MLOps

<img src="https://i.ibb.co/5RM26Cw/LOGO-COLOR2.png" width="500px">

Creado en [escueladedatosvivos.ai](https://escueladedatosvivos.ai) 🚀.

¿Consultas? En la página tenés soporte por IA guiada, comunidad y el acceso a certificación.

<br>

---  

# 1) Cargamos los datos

In [1]:
import numpy as np
import pandas as pd

In [2]:
# Para visualizar todas las columnas
pd.set_option('display.max_columns', None)

In [3]:
df_json_nested = pd.read_json('data/customers.json', lines=True)

In [4]:
print(df_json_nested.shape)
print(df_json_nested.columns)
display(df_json_nested.head(10))

(168, 5)
Index(['fraudulent', 'customer', 'orders', 'paymentMethods', 'transactions'], dtype='object')


,fraudulent,customer,orders,paymentMethods,transactions
0,False,"{'customerEmail': 'josephhoward@yahoo.com', 'c...","[{'orderId': 'vjbdvd', 'orderAmount': 18, 'ord...","[{'paymentMethodId': 'wt07xm68b', 'paymentMeth...","[{'transactionId': 'a9lcj51r', 'orderId': 'vjb..."
1,True,"{'customerEmail': 'evansjeffery@yahoo.com', 'c...","[{'orderId': 'nlghpa', 'orderAmount': 45, 'ord...","[{'paymentMethodId': 'y3xp697jx', 'paymentMeth...","[{'transactionId': '5mi94sfw', 'orderId': 'nlg..."
2,False,"{'customerEmail': 'andersonwilliam@yahoo.com',...","[{'orderId': 'yk34y2', 'orderAmount': 33, 'ord...","[{'paymentMethodId': '8pneoi03z', 'paymentMeth...","[{'transactionId': 'q3lyvbza', 'orderId': 'yk3..."
3,False,"{'customerEmail': 'rubenjuarez@yahoo.com', 'cu...","[{'orderId': 'fbz9ep', 'orderAmount': 34, 'ord...","[{'paymentMethodId': 'pdxjdwui4', 'paymentMeth...","[{'transactionId': 'vx4cjc27', 'orderId': 'fbz..."
4,True,"{'customerEmail': 'uchen@malone.com', 'custome...","[{'orderId': '56h7iw', 'orderAmount': 71, 'ord...","[{'paymentMethodId': 'w1i1zq3rg', 'paymentMeth...","[{'transactionId': 'q8j3dgni', 'orderId': '56h..."
5,True,"{'customerEmail': 'robinsoncynthia@dunn.com', ...","[{'orderId': 'vq7j5b', 'orderAmount': 25, 'ord...","[{'paymentMethodId': 'wferzj9en', 'paymentMeth...","[{'transactionId': '67qcpl0r', 'orderId': 'vq7..."
6,True,{'customerEmail': 'samuel15@saunders-rhodes.co...,"[{'orderId': 'sq7rk9', 'orderAmount': 40, 'ord...","[{'paymentMethodId': 'je76d0ulj', 'paymentMeth...","[{'transactionId': '9jutaun8', 'orderId': 'sq7..."
7,True,"{'customerEmail': 'johnlowery@gmail.com', 'cus...","[{'orderId': 'kanub7', 'orderAmount': 65, 'ord...","[{'paymentMethodId': '8xex97krx', 'paymentMeth...","[{'transactionId': 'cfoevtai', 'orderId': 'kan..."
8,False,"{'customerEmail': 'jameslandry@rodriguez.com',...","[{'orderId': 'mtg9kj', 'orderAmount': 25, 'ord...","[{'paymentMethodId': 'yk3gx9knx', 'paymentMeth...","[{'transactionId': 'idd9amrw', 'orderId': 'mtg..."
9,True,{'customerEmail': 'ubranch@rivera-parker.info'...,[],"[{'paymentMethodId': '37264k8rb', 'paymentMeth...",[]


In [5]:
display(df_json_nested.dtypes)

fraudulent          bool
customer          object
orders            object
paymentMethods    object
transactions      object
dtype: object

### Desanidamos las columnas y las mergeamos

In [6]:
rows = []
for idx, row in df_json_nested.iterrows():
    try:
        # Customers
        # Columns = ['customerEmail', 'customerPhone', 'customerDevice', 'customerIPAddress', 'customerBillingAddress']
        df_customers_ = pd.json_normalize(row['customer'])
        customers_dict = df_customers_.to_dict('records')[0]

        # Orders
        # Columns = ['orderId', 'orderAmount', 'orderState', 'orderShippingAddress']
        df_orders_nested_ = pd.DataFrame([md for md in row['orders']])

        # Payment Methods
        # Columns = ['paymentMethodId', 'paymentMethodRegistrationFailure', 'paymentMethodType', 'paymentMethodProvider', 'paymentMethodIssuer']
        df_payment_nested_ = pd.DataFrame([md for md in row['paymentMethods']])

        # Transactions
        # Columns = ['transactionId', 'orderId', 'paymentMethodId', 'transactionAmount', 'transactionFailed']
        df_transactions_nested_ = pd.DataFrame([md for md in row['transactions']])

        # Si no hay transacciones no hay ordenes
        if df_transactions_nested_.shape[0] == 0:
            payment_methods_list = df_payment_nested_.to_dict('records')
            if len(payment_methods_list) > 0:
                for pay_row in payment_methods_list:
                    rows.append({
                        'fraudulent': row['fraudulent'],
                        'customerEmail': customers_dict['customerEmail'],
                        'customerPhone': customers_dict['customerPhone'],
                        'customerDevice': customers_dict['customerDevice'],
                        'customerIPAddress': customers_dict['customerIPAddress'],
                        'customerBillingAddress': customers_dict['customerBillingAddress'],
                        'transactionId': np.nan,
                        'orderId': np.nan,
                        'paymentMethodId': pay_row['paymentMethodId'],
                        'transactionAmount': np.nan,
                        'transactionFailed': np.nan,
                        'orderAmount': np.nan,
                        'orderState': np.nan,
                        'orderShippingAddress': np.nan,
                        'paymentMethodRegistrationFailure': pay_row['paymentMethodRegistrationFailure'],
                        'paymentMethodType': pay_row['paymentMethodType'],
                        'paymentMethodProvider': pay_row['paymentMethodProvider'],
                        'paymentMethodIssuer': pay_row['paymentMethodIssuer'],
                    })
            else:
                rows.append({
                    'fraudulent': row['fraudulent'],
                    'customerEmail': customers_dict['customerEmail'],
                    'customerPhone': customers_dict['customerPhone'],
                    'customerDevice': customers_dict['customerDevice'],
                    'customerIPAddress': customers_dict['customerIPAddress'],
                    'customerBillingAddress': customers_dict['customerBillingAddress'],
                    'transactionId': np.nan,
                    'orderId': np.nan,
                    'paymentMethodId': np.nan,
                    'transactionAmount': np.nan,
                    'transactionFailed': np.nan,
                    'orderAmount': np.nan,
                    'orderState': np.nan,
                    'orderShippingAddress': np.nan,
                    'paymentMethodRegistrationFailure': np.nan,
                    'paymentMethodType': np.nan,
                    'paymentMethodProvider': np.nan,
                    'paymentMethodIssuer': np.nan,
                })
            continue
        else:
            # Hay transacciones
            df_transactions_full_nested_ = pd.merge(
                df_transactions_nested_,
                df_orders_nested_,
                how='left',
                on='orderId',
            ).merge(
                df_payment_nested_,
                how='left',
                on='paymentMethodId',
            )

            transactions_full_list = df_transactions_full_nested_.to_dict('records')
            for transaction_row in transactions_full_list:
                rows.append({
                    'fraudulent': row['fraudulent'],
                    'customerEmail': customers_dict['customerEmail'],
                    'customerPhone': customers_dict['customerPhone'],
                    'customerDevice': customers_dict['customerDevice'],
                    'customerIPAddress': customers_dict['customerIPAddress'],
                    'customerBillingAddress': customers_dict['customerBillingAddress'],
                    'transactionId': transaction_row['transactionId'],
                    'orderId': transaction_row['orderId'],
                    'paymentMethodId': transaction_row['paymentMethodId'],
                    'transactionAmount': transaction_row['transactionAmount'],
                    'transactionFailed': transaction_row['transactionFailed'],
                    'orderAmount': transaction_row['orderAmount'],
                    'orderState': transaction_row['orderState'],
                    'orderShippingAddress': transaction_row['orderShippingAddress'],
                    'paymentMethodRegistrationFailure': transaction_row['paymentMethodRegistrationFailure'],
                    'paymentMethodType': transaction_row['paymentMethodType'],
                    'paymentMethodProvider': transaction_row['paymentMethodProvider'],
                    'paymentMethodIssuer': transaction_row['paymentMethodIssuer'],
                })
    except Exception as exc:
        raise exc

In [7]:
len(rows)

653

In [8]:
df_data = pd.DataFrame(rows)

In [9]:
print(df_data.shape)
print(df_data.columns)
display(df_data.head(5))

(653, 18)
Index(['fraudulent', 'customerEmail', 'customerPhone', 'customerDevice',
       'customerIPAddress', 'customerBillingAddress', 'transactionId',
       'orderId', 'paymentMethodId', 'transactionAmount', 'transactionFailed',
       'orderAmount', 'orderState', 'orderShippingAddress',
       'paymentMethodRegistrationFailure', 'paymentMethodType',
       'paymentMethodProvider', 'paymentMethodIssuer'],
      dtype='object')


,fraudulent,customerEmail,customerPhone,customerDevice,customerIPAddress,customerBillingAddress,transactionId,orderId,paymentMethodId,transactionAmount,transactionFailed,orderAmount,orderState,orderShippingAddress,paymentMethodRegistrationFailure,paymentMethodType,paymentMethodProvider,paymentMethodIssuer
0,False,josephhoward@yahoo.com,400-108-5415,yyeiaxpltf82440jnb3v,8.129.104.40,"5493 Jones Islands\nBrownside, CA 51896",a9lcj51r,vjbdvd,wt07xm68b,18.0,False,18.0,pending,"5493 Jones Islands\nBrownside, CA 51896",True,card,JCB 16 digit,Citizens First Banks
1,False,josephhoward@yahoo.com,400-108-5415,yyeiaxpltf82440jnb3v,8.129.104.40,"5493 Jones Islands\nBrownside, CA 51896",y4wcv03i,yp6x27,wt07xm68b,26.0,False,26.0,fulfilled,"5493 Jones Islands\nBrownside, CA 51896",True,card,JCB 16 digit,Citizens First Banks
2,True,evansjeffery@yahoo.com,1-788-091-7546,r0jpm7xaeqqa3kr6mzum,219.173.211.202,356 Elizabeth Inlet Suite 120\nPort Joshuabury...,5mi94sfw,nlghpa,41ug157xz,45.0,False,45.0,fulfilled,"898 Henry Ports\nNew Keithview, CA 95893-2497",False,bitcoin,American Express,Bastion Banks
3,True,evansjeffery@yahoo.com,1-788-091-7546,r0jpm7xaeqqa3kr6mzum,219.173.211.202,356 Elizabeth Inlet Suite 120\nPort Joshuabury...,br8ba1nu,uw0eeb,41ug157xz,23.0,False,23.0,fulfilled,356 Elizabeth Inlet Suite 120\nPort Joshuabury...,False,bitcoin,American Express,Bastion Banks
4,True,evansjeffery@yahoo.com,1-788-091-7546,r0jpm7xaeqqa3kr6mzum,219.173.211.202,356 Elizabeth Inlet Suite 120\nPort Joshuabury...,a33145ss,bn44oh,y3xp697jx,43.0,True,43.0,fulfilled,"5093 Bryan Forks\nJoshuaton, FM 01565-9801",True,bitcoin,VISA 16 digit,Solace Banks


# 2) Transformación de columnas

Vamos a transformar los valores de algunas columnas que parecen únicos para que sean utilizables.

In [10]:
import funpymodeling

In [11]:
funpymodeling.status(df_data)

,variable,q_nan,p_nan,q_zeros,p_zeros,unique,type
0,fraudulent,0,0.000000,386,0.591118,2,bool
1,customerEmail,0,0.000000,0,0.000000,161,object
2,customerPhone,0,0.000000,0,0.000000,168,object
3,customerDevice,0,0.000000,0,0.000000,168,object
4,customerIPAddress,0,0.000000,0,0.000000,165,object
5,customerBillingAddress,0,0.000000,0,0.000000,166,object
6,transactionId,30,0.045942,0,0.000000,623,object
7,orderId,30,0.045942,0,0.000000,478,object
8,paymentMethodId,12,0.018377,0,0.000000,255,object
9,transactionAmount,30,0.045942,0,0.000000,67,float64


## 2.1) customerEmail

Generalmente solemos recibir mail de clientes que utilizan servicios de mensajería como `@hotmail`, `@yahoo` y `@gmail`. Esos son los tipos de mail más comunes.

Muchas veces ciertos clientes con intenciones maliciosas usan otro tipo de mail como ser `@u6n7x`, `@1jcfcxs7` o `@6eph`.

In [12]:
# Si email = "edvai@yahoo.com"
# ==> aux = yahoo.com
# ==> mail = yahoo
# ==> domain = com

popular_providers = ['yahoo', 'gmail', 'hotmail']

email_domains = []
email_provider = []
for email in df_data['customerEmail']:
    try:
        aux = email.split('@')[1]
        domain = aux.split('.')[1]
        mail = aux.split('.')[0]
        
        if mail in popular_providers:
            email_provider.append(mail)
        else:
            email_provider.append('other')

        email_domains.append(domain)
    except Exception as exc:
        email_domains.append('weird')
        email_provider.append('weird')

In [13]:
df_data['emailDomain'] = email_domains
df_data['emailProvider'] = email_provider

Eliminamos la columna `customerEmail`

In [14]:
df_data = df_data.drop(['customerEmail'], axis=1)

In [15]:
df_data.head(5)

,fraudulent,customerPhone,customerDevice,customerIPAddress,customerBillingAddress,transactionId,orderId,paymentMethodId,transactionAmount,transactionFailed,orderAmount,orderState,orderShippingAddress,paymentMethodRegistrationFailure,paymentMethodType,paymentMethodProvider,paymentMethodIssuer,emailDomain,emailProvider
0,False,400-108-5415,yyeiaxpltf82440jnb3v,8.129.104.40,"5493 Jones Islands\nBrownside, CA 51896",a9lcj51r,vjbdvd,wt07xm68b,18.0,False,18.0,pending,"5493 Jones Islands\nBrownside, CA 51896",True,card,JCB 16 digit,Citizens First Banks,com,yahoo
1,False,400-108-5415,yyeiaxpltf82440jnb3v,8.129.104.40,"5493 Jones Islands\nBrownside, CA 51896",y4wcv03i,yp6x27,wt07xm68b,26.0,False,26.0,fulfilled,"5493 Jones Islands\nBrownside, CA 51896",True,card,JCB 16 digit,Citizens First Banks,com,yahoo
2,True,1-788-091-7546,r0jpm7xaeqqa3kr6mzum,219.173.211.202,356 Elizabeth Inlet Suite 120\nPort Joshuabury...,5mi94sfw,nlghpa,41ug157xz,45.0,False,45.0,fulfilled,"898 Henry Ports\nNew Keithview, CA 95893-2497",False,bitcoin,American Express,Bastion Banks,com,yahoo
3,True,1-788-091-7546,r0jpm7xaeqqa3kr6mzum,219.173.211.202,356 Elizabeth Inlet Suite 120\nPort Joshuabury...,br8ba1nu,uw0eeb,41ug157xz,23.0,False,23.0,fulfilled,356 Elizabeth Inlet Suite 120\nPort Joshuabury...,False,bitcoin,American Express,Bastion Banks,com,yahoo
4,True,1-788-091-7546,r0jpm7xaeqqa3kr6mzum,219.173.211.202,356 Elizabeth Inlet Suite 120\nPort Joshuabury...,a33145ss,bn44oh,y3xp697jx,43.0,True,43.0,fulfilled,"5093 Bryan Forks\nJoshuaton, FM 01565-9801",True,bitcoin,VISA 16 digit,Solace Banks,com,yahoo


## 2.2) customerIPAddress

Por ejemplo, nuestro localhost es `127.0.0.1`

Según el protocolo, podemos llegar a tener una subnet mask `255.255.255.255` (la máxima cantidad de caracteres numéricos), es decir podes alcanzar una longitud 15 caracteres.

In [16]:
ip_addresses = []
for address in df_data['customerIPAddress']:
    # A veces lo toma como flotante al address
    aux_address = str(address)
    if len(aux_address) > 15:
        ip_addresses.append('digits_and_letters')
    else:
        ip_addresses.append('only_letters')

In [17]:
df_data['customerIPAddressSimplified'] = ip_addresses

Eliminamos la columna `customerIPAddress`

In [18]:
df_data = df_data.drop(['customerIPAddress'], axis=1)

In [19]:
df_data.head(5)

,fraudulent,customerPhone,customerDevice,customerBillingAddress,transactionId,orderId,paymentMethodId,transactionAmount,transactionFailed,orderAmount,orderState,orderShippingAddress,paymentMethodRegistrationFailure,paymentMethodType,paymentMethodProvider,paymentMethodIssuer,emailDomain,emailProvider,customerIPAddressSimplified
0,False,400-108-5415,yyeiaxpltf82440jnb3v,"5493 Jones Islands\nBrownside, CA 51896",a9lcj51r,vjbdvd,wt07xm68b,18.0,False,18.0,pending,"5493 Jones Islands\nBrownside, CA 51896",True,card,JCB 16 digit,Citizens First Banks,com,yahoo,only_letters
1,False,400-108-5415,yyeiaxpltf82440jnb3v,"5493 Jones Islands\nBrownside, CA 51896",y4wcv03i,yp6x27,wt07xm68b,26.0,False,26.0,fulfilled,"5493 Jones Islands\nBrownside, CA 51896",True,card,JCB 16 digit,Citizens First Banks,com,yahoo,only_letters
2,True,1-788-091-7546,r0jpm7xaeqqa3kr6mzum,356 Elizabeth Inlet Suite 120\nPort Joshuabury...,5mi94sfw,nlghpa,41ug157xz,45.0,False,45.0,fulfilled,"898 Henry Ports\nNew Keithview, CA 95893-2497",False,bitcoin,American Express,Bastion Banks,com,yahoo,only_letters
3,True,1-788-091-7546,r0jpm7xaeqqa3kr6mzum,356 Elizabeth Inlet Suite 120\nPort Joshuabury...,br8ba1nu,uw0eeb,41ug157xz,23.0,False,23.0,fulfilled,356 Elizabeth Inlet Suite 120\nPort Joshuabury...,False,bitcoin,American Express,Bastion Banks,com,yahoo,only_letters
4,True,1-788-091-7546,r0jpm7xaeqqa3kr6mzum,356 Elizabeth Inlet Suite 120\nPort Joshuabury...,a33145ss,bn44oh,y3xp697jx,43.0,True,43.0,fulfilled,"5093 Bryan Forks\nJoshuaton, FM 01565-9801",True,bitcoin,VISA 16 digit,Solace Banks,com,yahoo,only_letters


## 2.3) customerBillingAddress vrs. orderShippingAddress

In [20]:
# Si address = "5493 Jones Islands\nBrownside, CA 51896"
# ==> aux = "Brownside, CA 51896"
# ==> city_with_number = "CA 51896"
# ==> city = "CA"

def extract_city_name(name_column):
    city_list = []
    for address in df_data[name_column]:
        try:
            aux = address.split('\n')[1]
            city_with_number = aux.split(', ')[1]
            city = city_with_number.split(' ')[0]

            city_list.append(city)
        except Exception as exc:
            # Sino aparece el nombre de la ciudad asignamos: unknown
            city_list.append("unknown")

    return city_list

In [21]:
df_data["customerBillingAddress"] = extract_city_name("customerBillingAddress")
df_data["orderShippingAddress"] = extract_city_name("orderShippingAddress")

In [22]:
df_data.head(5)

,fraudulent,customerPhone,customerDevice,customerBillingAddress,transactionId,orderId,paymentMethodId,transactionAmount,transactionFailed,orderAmount,orderState,orderShippingAddress,paymentMethodRegistrationFailure,paymentMethodType,paymentMethodProvider,paymentMethodIssuer,emailDomain,emailProvider,customerIPAddressSimplified
0,False,400-108-5415,yyeiaxpltf82440jnb3v,CA,a9lcj51r,vjbdvd,wt07xm68b,18.0,False,18.0,pending,CA,True,card,JCB 16 digit,Citizens First Banks,com,yahoo,only_letters
1,False,400-108-5415,yyeiaxpltf82440jnb3v,CA,y4wcv03i,yp6x27,wt07xm68b,26.0,False,26.0,fulfilled,CA,True,card,JCB 16 digit,Citizens First Banks,com,yahoo,only_letters
2,True,1-788-091-7546,r0jpm7xaeqqa3kr6mzum,NM,5mi94sfw,nlghpa,41ug157xz,45.0,False,45.0,fulfilled,CA,False,bitcoin,American Express,Bastion Banks,com,yahoo,only_letters
3,True,1-788-091-7546,r0jpm7xaeqqa3kr6mzum,NM,br8ba1nu,uw0eeb,41ug157xz,23.0,False,23.0,fulfilled,NM,False,bitcoin,American Express,Bastion Banks,com,yahoo,only_letters
4,True,1-788-091-7546,r0jpm7xaeqqa3kr6mzum,NM,a33145ss,bn44oh,y3xp697jx,43.0,True,43.0,fulfilled,FM,True,bitcoin,VISA 16 digit,Solace Banks,com,yahoo,only_letters


También podemos evaluar que la ciudad de `customerBillingAddress` y `orderShippingAddress` coinciden.

Vamos a considerar si una de las 2 address al momento de comparar es `unknown`, le asignaremos como resultado `unknown`.

Si ambas coinciden asignaremos `yes` y caso contrario `no`.

In [23]:
same_city = []
for idx in range(len(df_data)):
    billing_address = df_data["customerBillingAddress"][idx]
    order_address = df_data["orderShippingAddress"][idx]

    response = "unknown"
    if (billing_address != "unknown") and (order_address != "unknown"):
        if billing_address == order_address:
            response = "yes"
        else:
            response = "no"

    same_city.append(response)

In [24]:
df_data["sameCity"] = same_city

Con las columnas `customerBillingAddress` y `orderShippingAddress` se podría hacer un diagrama de flujo o de conexiones geoespacial o representarlo en un mapa.

Conforme nuestra cartera de clientes crezca y tengamos más ciudades, más posibles valores tomarán las columnas `customerBillingAddress` y `orderShippingAddress`.

En cambio `sameCity` va a seguir teniendo los mismos 3 posibles valores.

Bajo ese criterio  vamos a despreciar ambas columnas y solo conservaremos `sameCity`.

In [25]:
df_data = df_data.drop(["customerBillingAddress", "orderShippingAddress"], axis=1)

In [26]:
df_data.head(5)

,fraudulent,customerPhone,customerDevice,transactionId,orderId,paymentMethodId,transactionAmount,transactionFailed,orderAmount,orderState,paymentMethodRegistrationFailure,paymentMethodType,paymentMethodProvider,paymentMethodIssuer,emailDomain,emailProvider,customerIPAddressSimplified,sameCity
0,False,400-108-5415,yyeiaxpltf82440jnb3v,a9lcj51r,vjbdvd,wt07xm68b,18.0,False,18.0,pending,True,card,JCB 16 digit,Citizens First Banks,com,yahoo,only_letters,yes
1,False,400-108-5415,yyeiaxpltf82440jnb3v,y4wcv03i,yp6x27,wt07xm68b,26.0,False,26.0,fulfilled,True,card,JCB 16 digit,Citizens First Banks,com,yahoo,only_letters,yes
2,True,1-788-091-7546,r0jpm7xaeqqa3kr6mzum,5mi94sfw,nlghpa,41ug157xz,45.0,False,45.0,fulfilled,False,bitcoin,American Express,Bastion Banks,com,yahoo,only_letters,no
3,True,1-788-091-7546,r0jpm7xaeqqa3kr6mzum,br8ba1nu,uw0eeb,41ug157xz,23.0,False,23.0,fulfilled,False,bitcoin,American Express,Bastion Banks,com,yahoo,only_letters,yes
4,True,1-788-091-7546,r0jpm7xaeqqa3kr6mzum,a33145ss,bn44oh,y3xp697jx,43.0,True,43.0,fulfilled,True,bitcoin,VISA 16 digit,Solace Banks,com,yahoo,only_letters,no


## 2.4) customerPhone

El número de teléfono prácticamente es un identificador único.

Pero lo que se podría hacer es obtener el código de área para saber dónde está registrado el celular. 

**Por ejemplo**

Un número "+54 9 261-1234-578", se podría extraer el primer número y buscar de qué país es. En este caso el +54 corresponde a Argentina y el 261 a la provincia de Mendoza.

Y de esa manera se podría corroborar que el número del usuario tiene una correspondencia con el país o ciudad donde reside la persona.

En este caso hay un universo de posibilidades en cómo se presenta el dato en `customerPhone` y no registra un patrón. Si tuvieramos más contexto podríamos conservar la columna.

Por este motivo vamos a eliminar la columna:

In [27]:
df_data = df_data.drop(['customerPhone'], axis=1)

In [28]:
df_data.head(5)

,fraudulent,customerDevice,transactionId,orderId,paymentMethodId,transactionAmount,transactionFailed,orderAmount,orderState,paymentMethodRegistrationFailure,paymentMethodType,paymentMethodProvider,paymentMethodIssuer,emailDomain,emailProvider,customerIPAddressSimplified,sameCity
0,False,yyeiaxpltf82440jnb3v,a9lcj51r,vjbdvd,wt07xm68b,18.0,False,18.0,pending,True,card,JCB 16 digit,Citizens First Banks,com,yahoo,only_letters,yes
1,False,yyeiaxpltf82440jnb3v,y4wcv03i,yp6x27,wt07xm68b,26.0,False,26.0,fulfilled,True,card,JCB 16 digit,Citizens First Banks,com,yahoo,only_letters,yes
2,True,r0jpm7xaeqqa3kr6mzum,5mi94sfw,nlghpa,41ug157xz,45.0,False,45.0,fulfilled,False,bitcoin,American Express,Bastion Banks,com,yahoo,only_letters,no
3,True,r0jpm7xaeqqa3kr6mzum,br8ba1nu,uw0eeb,41ug157xz,23.0,False,23.0,fulfilled,False,bitcoin,American Express,Bastion Banks,com,yahoo,only_letters,yes
4,True,r0jpm7xaeqqa3kr6mzum,a33145ss,bn44oh,y3xp697jx,43.0,True,43.0,fulfilled,True,bitcoin,VISA 16 digit,Solace Banks,com,yahoo,only_letters,no


## 2.5) Identificadores únicos

Muchas veces en este tipo de productos de prevención de fraude se busca reducir la fricción. Por eso tenemos distintos identificadores del dispositivo del usuario, tracking del producto, etc.

En este caso tenemos identificadores únicos en las columnas `customerDevice`, `orderId`, `transactionId` y `paymentMethodId`.

Por esa razón los vamos a eliminar.

In [29]:
columns = ['customerDevice', 'orderId', 'transactionId', 'paymentMethodId']
df_data = df_data.drop(columns, axis=1)

In [30]:
df_data.head(5)

,fraudulent,transactionAmount,transactionFailed,orderAmount,orderState,paymentMethodRegistrationFailure,paymentMethodType,paymentMethodProvider,paymentMethodIssuer,emailDomain,emailProvider,customerIPAddressSimplified,sameCity
0,False,18.0,False,18.0,pending,True,card,JCB 16 digit,Citizens First Banks,com,yahoo,only_letters,yes
1,False,26.0,False,26.0,fulfilled,True,card,JCB 16 digit,Citizens First Banks,com,yahoo,only_letters,yes
2,True,45.0,False,45.0,fulfilled,False,bitcoin,American Express,Bastion Banks,com,yahoo,only_letters,no
3,True,23.0,False,23.0,fulfilled,False,bitcoin,American Express,Bastion Banks,com,yahoo,only_letters,yes
4,True,43.0,True,43.0,fulfilled,True,bitcoin,VISA 16 digit,Solace Banks,com,yahoo,only_letters,no


In [31]:
funpymodeling.status(df_data)

,variable,q_nan,p_nan,q_zeros,p_zeros,unique,type
0,fraudulent,0,0.000000,386,0.591118,2,bool
1,transactionAmount,30,0.045942,0,0.000000,67,float64
2,transactionFailed,30,0.045942,455,0.696784,2,object
3,orderAmount,30,0.045942,0,0.000000,67,float64
4,orderState,30,0.045942,0,0.000000,3,object
5,paymentMethodRegistrationFailure,12,0.018377,559,0.856049,2,object
6,paymentMethodType,12,0.018377,0,0.000000,4,object
7,paymentMethodProvider,12,0.018377,0,0.000000,10,object
8,paymentMethodIssuer,12,0.018377,0,0.000000,20,object
9,emailDomain,0,0.000000,0,0.000000,6,object


# 3) Guardar dataset como CSV

In [32]:
df_data.to_csv("data/customer_dataset.csv", index=False, sep=',')